In [1]:
import redshift_connector
import pandas as pd
import os

In [2]:
host = os.environ['host']
database = os.environ['database']
user = os.environ['user']
password = os.environ['password']
schema = os.environ['schema']

In [3]:
def redshift_query(host : str, database : str, user : str, password : str, query: str ) -> pd.DataFrame:
    conn = redshift_connector.connect(
     host=host,
     database=database,
     user=user,
     password=password
    )
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetch_dataframe()
    
    return result

1.	Our client Barilla would like to know how many customers purchased their pasta products during February 2020. 

In [125]:
query = f"""
SELECT
C.client,
C.brand,
COUNT(DISTINCT G.member_id) 
FROM {schema}.groceryorders AS G
LEFT JOIN {schema}.clientbrand AS C
ON G.brand = C.brand
WHERE
UPPER(C.client) = 'BARILLA'
AND
order_date BETWEEN '2020-02-01' AND '2020-02-29'
AND
UPPER(product_category) = 'PASTA'
GROUP BY
C.client,
C.brand
"""

In [126]:
result = redshift_query(host, database, user, password, query)
result

,client,brand,count
0,Barilla,Barilla,429


2. We would like to present the monthly grocery sales for each vendor and provide the sales in dollars and also as percentages. In addition, the following vendors should be transformed as kroger: kingSoopers, smiths, fredMeyer, frys, picknsave & ralphs. The following vendor should be considered as albertsons: safeway. 

In [89]:
query = f"""
SELECT
CASE
    WHEN vendor = 'kingSoopers' THEN 'kroger'
    WHEN vendor = 'smiths' THEN 'kroger'
    WHEN vendor = 'fredMeyer' THEN 'kroger'
    WHEN vendor = 'frys' THEN 'kroger'
    WHEN vendor = 'picknsave' THEN 'kroger'
    WHEN vendor = 'ralphs' THEN 'kroger'
    WHEN vendor = 'safeway' THEN 'albertsons'
    ELSE vendor
END AS group_vendors,
to_char(order_date, 'MM') as month_order,
to_char(order_date, 'YYYY') as year_order,
SUM(cost) as total_cost,
SUM(cost) * 100 / SUM(SUM(cost)) OVER(PARTITION BY month_order,year_order ) AS percentage_cost
FROM {schema}.groceryorders
GROUP BY
group_vendors,
month_order,
year_order
"""

In [90]:
result = redshift_query(host, database, user, password, query)
result

,group_vendors,month_order,year_date,total_cost,percentage_cost
0,instacart,02,2020,NaN,NaN
1,walmartGrocery,02,2020,8347.59,81.858780
2,kroger,02,2020,1164.14,11.415879
3,shipt,02,2020,563.12,5.522111
4,amazonFresh,02,2020,122.70,1.203230
...,...,...,...,...,...
75,amazonFresh,07,2020,33.74,0.527143
76,walmartGrocery,07,2020,4676.81,73.068991
77,instacart,07,2020,273.63,4.275108
78,kroger,07,2020,1239.30,19.362429


3. Our meal team is looking to create a budget friendly meal plan and would like to avoid using expensive recipe ingredients. They would like to know the top 10 least expensive products our customers have purchased. 

In [110]:
query = f"""
WITH temp_table AS(
    SELECT
    brand,
    product_category,
    product_name,
    cost / quantity as unit_cost
    FROM {schema}.groceryorders
    WHERE
    quantity > 0
    AND
    cost > 0
    order by
    unit_cost ASC
),
avg_table AS (
    SELECT
    brand,
    product_category,
    product_name,
    AVG(unit_cost) as avg_unit_cost,
    RANK () OVER (ORDER BY avg_unit_cost ASC) AS rnk_avg_cost
    FROM temp_table
    GROUP BY
    brand,
    product_category,
    product_name
    ORDER BY
    avg_unit_cost ASC
)
SELECT
brand,
product_category,
product_name,
avg_unit_cost
FROM avg_table
WHERE
rnk_avg_cost <= 10
"""

In [111]:
result = redshift_query(host, database, user, password, query)
result

,brand,product_category,product_name,avg_unit_cost
0,Swanson,broth,swanson natural goodness chicken broth 14.5 oz...,0.570000
1,Hillshire,ham,Carolina Pride Honey Ham,0.620000
2,Swanson,broth,swansons swanson natural goodness chicken brot...,0.675000
3,Swanson,broth,swanson 50% less sodium beef broth 14.5 oz.,0.756667
4,Swanson,broth,swanson chicken broth 14.5 oz. can,0.782941
5,Swanson,broth,swanson vegetable broth 14.5 oz. can,0.790000
6,Swanson,broth,swanson 50% less sodium beef broth 14.5 oz. can,0.815000
7,Swanson,broth,Swanson50% Less Sodium Beef Broth,0.830356
8,Swanson,broth,Swanson Clear Beef Broth,0.840866
9,Red Cap,seasoning,mccormick brown gravy mix 0.87 oz,0.850000


4. The brand Tyson is considering partnering with eMeals to promote chicken products. Tyson would like to know what their current market share of online chicken sales and who are their competitors.

In [77]:
query = f'''
SELECT  
client,
UPPER(brand) as brand,
active
FROM {schema}.clientbrand
ORDER BY
active DESC
'''

brands = redshift_query(host, database, user, password, query)
brands

,client,brand,active
0,Treasury Wine,STAG LEAP,True
1,Gortons,GORTONS,True
2,EJ Gallo Winery,BAREFOOT,True
3,Brown-Forman,JACK DANIEL,True
4,Treasury Wine,CHATEAU ST JEAN,True
5,Lactalis,PRESIDENT,True
6,EJ Gallo Winery,ECCO DOMANI,True
7,EJ Gallo Winery,CLOS DU BOIS,True
8,Barilla,BARILLA,True
9,EJ Gallo Winery,LA MARCA,True


In [109]:
query = f"""
SELECT
CAST(to_char(order_date, 'MM') AS int) month_order,
CAST(to_char(order_date, 'YYYY') AS INT) as year_order,
brand,
product_category,
UPPER(product_name) AS product_name,
SUM(cost) as total_cost
FROM {schema}.groceryorders
WHERE
UPPER(product_category) = 'CHICKEN'
GROUP BY
month_order,
year_order,
brand,
product_category,
product_name
"""

In [115]:
result = redshift_query(host, database, user, password, query)
result

,month_order,year_order,brand,product_category,product_name,total_cost
0,1,2020,None,chicken,TYSON ALL NATURAL* BONELESS SKINLESS CHICKEN T...,550.68
1,1,2020,None,chicken,TYSON TRIMMED & READY FRESH BONELESS SKINLESS ...,2284.08
2,1,2020,None,chicken,TYSON BONELESS SKINLESS CHICKEN BREAST TENDERL...,854.77
3,1,2020,None,chicken,DEL REAL FOODS SHREDDED CHICKEN,55.36
4,1,2020,None,chicken,TYSON BONELESS SKINLESS CHICKEN BREASTS,1085.27
...,...,...,...,...,...,...
205,1,2020,None,chicken,SIMPLE TRUTH NATURAL BONELESS & SKINLESS CHICK...,NaN
206,1,2020,None,chicken,FARM RICH TIME OUTS BBQ CHICKEN BIT,7.56
207,1,2020,None,chicken,"SMART CHICKEN ORGANIC CHICKEN BREAST, BONELESS...",13.33
208,1,2020,None,chicken,PUBLIX BONELESS SKINLESS CHICKEN THIGHS,7.63


In [118]:
for brand in brands['brand']:
    result['brand'] = result.apply(lambda x: brand if brand in x['product_name'] else x['brand']  , axis = 1 )

In [119]:
result

,month_order,year_order,brand,product_category,product_name,total_cost
0,1,2020,TYSON,chicken,TYSON ALL NATURAL* BONELESS SKINLESS CHICKEN T...,550.68
1,1,2020,TYSON,chicken,TYSON TRIMMED & READY FRESH BONELESS SKINLESS ...,2284.08
2,1,2020,TYSON,chicken,TYSON BONELESS SKINLESS CHICKEN BREAST TENDERL...,854.77
3,1,2020,Not Found,chicken,DEL REAL FOODS SHREDDED CHICKEN,55.36
4,1,2020,TYSON,chicken,TYSON BONELESS SKINLESS CHICKEN BREASTS,1085.27
...,...,...,...,...,...,...
205,1,2020,Not Found,chicken,SIMPLE TRUTH NATURAL BONELESS & SKINLESS CHICK...,NaN
206,1,2020,Not Found,chicken,FARM RICH TIME OUTS BBQ CHICKEN BIT,7.56
207,1,2020,Not Found,chicken,"SMART CHICKEN ORGANIC CHICKEN BREAST, BONELESS...",13.33
208,1,2020,Not Found,chicken,PUBLIX BONELESS SKINLESS CHICKEN THIGHS,7.63


In [125]:
result_agg = result.groupby(by = ['month_order', 'year_order', 'brand']).agg({'total_cost' : sum})

In [127]:
result_agg.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

total_cost
month_order year_order brand                
1           2020       HORMEL       0.010867
                       Not Found   36.300788
                       TYSON       63.688345
2           2020       Not Found  100.000000

5. A potential client would like to see the month-over-month change in wine sales as a percentage.

In [185]:
query = f"""
WITH lag_table AS(
SELECT
UPPER(product_category) AS product_category,
CAST(to_char(order_date, 'MM') AS int) month_order,
CAST(to_char(order_date, 'YYYY') AS INT) as year_order,
SUM(cost) as current_month,
LAG(current_month) OVER (PARTITION BY year_order ORDER BY year_order, month_order ASC ) AS previous_month,
current_month / previous_month - 1 AS diff
FROM {schema}.groceryorders
WHERE
UPPER(product_category) = 'WINE'
GROUP BY
product_category,
month_order,
year_order
)
SELECT
product_category,
month_order,
year_order,
diff
FROM
lag_table
"""

In [186]:
result = redshift_query(host, database, user, password, query)
result

,product_category,month_order,year_order,diff
0,WINE,12,2020,0.260522
1,WINE,11,2020,0.253437
2,WINE,10,2020,-0.064586
3,WINE,9,2020,0.086648
4,WINE,8,2020,-0.303858
5,WINE,7,2020,0.318119
6,WINE,6,2020,-0.087548
7,WINE,5,2020,0.749365
8,WINE,4,2020,-0.508381
9,WINE,3,2020,-0.172375


6. Our meal team is looking to use wine as recipe ingredient in order to help promote a brand and increase brand sales. They would like to know which brand to use in future recipes.

In [218]:
query = f"""
WITH participation_table AS (
SELECT
brand,
to_char(order_date, 'MM') as month_order,
to_char(order_date, 'YYYY') as year_order,
SUM(cost) as total_cost,
SUM(cost) * 100 / SUM(SUM(cost)) OVER(PARTITION BY month_order,year_order ) AS current_participation
FROM {schema}.groceryorders
WHERE
UPPER(product_category) = 'WINE'
GROUP BY
brand,
month_order,
year_order
),
lag_table AS(
    SELECT
    brand,
    month_order,
    year_order,
    total_cost,
    current_participation,    
    LAG(current_participation) OVER (PARTITION BY brand, year_order ORDER BY brand, year_order, month_order ASC ) AS previous_participation
    FROM participation_table
    GROUP BY
    brand,
    month_order,
    year_order,
    total_cost,
    current_participation
)

SELECT
brand,
month_order,
year_order,
current_participation / previous_participation - 1 as diff
FROM lag_table
WHERE
diff IS NOT NULL
ORDER BY
year_order ASC,
month_order ASC,
diff DESC
"""

In [219]:
result = redshift_query(host, database, user, password, query)
result

,brand,month_order,year_order,diff
0,Black Box,02,2020,1.842563
1,Barefoot,02,2020,0.195752
2,Woodbridge,02,2020,0.046746
3,Robert Mondavi Private Selection,02,2020,-0.071567
4,Kendall-Jackson,02,2020,-0.207881
...,...,...,...,...
98,Barefoot,02,2021,-0.336260
99,Apothic,02,2021,-0.339460
100,19 Crimes,02,2021,-0.413935
101,Mirassou,02,2021,-0.433941


7. A potential client would like to understand the shopping frequency of eMeals customers. The client is interested in knowing what percentage of customers place online grocery orders more than twice a month. They also would like to know what percentage of customers order from different vendors.

In [44]:
query = f"""
WITH count_table AS(
SELECT
to_char(order_date, 'MM') as month_order,
to_char(order_date, 'YYYY') as year_order,
member_id,
COUNT(order_id) as qty_orders,
COUNT(DISTINCT vendor) as qty_vendors
FROM {schema}.groceryorders
GROUP BY
month_order,
year_order,
member_id
),
group_table AS (
SELECT
month_order,
year_order,
SUM(CASE WHEN qty_orders >= 2 THEN 1 ELSE 0 END) AS qty_order_gt_2,
COUNT(member_id) AS customers,
SUM(CASE WHEN qty_vendors >= 2 THEN 1 ELSE 0 END) AS qty_vendor_gt_2
FROM count_table
GROUP BY
month_order,
year_order)

SELECT
month_order,
year_order,
customers,
qty_order_gt_2,
qty_vendor_gt_2,
CAST(qty_order_gt_2 AS DECIMAL) / customers AS customers_with_order_gt_2,
CAST(qty_vendor_gt_2 AS DECIMAL) / customers AS customers_with_vendor_gt_2
FROM
group_table
GROUP BY
month_order,
year_order,
customers,
qty_order_gt_2,
qty_vendor_gt_2
"""

In [45]:
result = redshift_query(host, database, user, password, query)
result

,month_order,year_order,customers,qty_order_gt_2,qty_vendor_gt_2,customers_with_order_gt_2,customers_with_vendor_gt_2
0,01,2020,3917,1100,11,0.28082716364564717896,0.00280827163645647178
1,03,2020,1253,197,4,0.15722266560255387071,0.00319233838786911412
2,08,2020,2324,463,8,0.19922547332185886402,0.00344234079173838209
3,09,2020,2118,395,10,0.18649669499527856468,0.00472143531633616619
4,11,2020,1718,339,7,0.19732246798603026775,0.00407450523864959254
5,12,2020,1429,266,6,0.18614415675297410776,0.00419874037788663400
6,02,2021,1229,124,1,0.10089503661513425549,0.00081366965012205044
7,05,2020,1076,173,2,0.16078066914498141263,0.00185873605947955390
8,07,2020,1543,245,4,0.15878159429682436811,0.00259235255994815294
9,01,2021,2195,274,6,0.12482915717539863325,0.00273348519362186788


8.	Our team is interested in understanding how customer engagement leads to a customer completing an online grocery order. One engagement metric is the percentage of customers who viewed a meal and then completed an order.

In [57]:
query = f"""
SELECT 
V.member_id,
V.recipe_title,
V.views,
S.event
FROM {schema}.app_views AS V
INNER JOIN {schema}.app_added_shopping_list_meal AS S
ON V.member_id = S.member_id AND V.recipe_title = S.recipe_title


"""

In [64]:
result = redshift_query(host, database, user, password, query)
result

,member_id
0,None


9.	As we continue to understand customer engagement, we would like for our warehouse to include aggregated data on customers who added a meal to their shopping list. Since the source data is already in the data warehouse, we will use a stored procedure to perform a nightly ETL of data. Please provide stored procedure.</br>
a.	Source is the app event tracking table:  app_added_shopping_list_meal</br>
b.	Destination (target) is the reporting table:  app_adds


10.	Our data scientist is looking to perform a cohort analysis and would like your help building the initial solution. Please have python connect to data warehouse and create dataframe of customer, start date and their total views, adds, orders and sales. Please provide python solution.

In [65]:
query = f"""
SELECT 
V.member_id,
SUM(V.views) as total_views,
COUNT(G.order_id) as total_orders
FROM {schema}.app_views AS V
INNER JOIN {schema}.groceryorders AS G
ON V.member_id = G.member_id 
GROUP BY
V.member_id
"""

In [66]:
result = redshift_query(host, database, user, password, query)
result

,member_id,total_views,total_orders
0,1063049,16,16
1,1141929,98,92
2,1066284,252,232
3,1093747,60,60
4,1196938,72,66
...,...,...,...
15205,861267,2,1
15206,1426279,2,2
15207,1415555,2,1
15208,482103,3,3
